# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

### Структура данных:

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

### План исследования

```
1) Изучить файл с данными.
2) Разбить данные на три выборки: обучающая, валидационная и тестовая.
3) Исследовать три модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.
4) Найти оптимальные параметры для каждой модели и выбрать одну из них для обучения модели.
5) Оценить точность обученной модели.
6) Оценить адекватность модели.
```

In [203]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None 

## 1 - Откройте и изучите файл

In [204]:
df = pd.read_csv('/Users/yuliabezginova/Documents/DS/praktikum/project-5_ML_mobile-learning/users_behavior.csv')

In [205]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [206]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Вывод:

Перед нами стоит задача ___классификации___. За целевой признак берем столбец _is_ultra_. Остальные признаки помогут нам предсказывать решение по столбцу _is_ultra_. В датасете 3200 наблюдений, это не так много, а значит, может возникнуть проблема _underfitting_: если модель будет хорошо работать на обучающем наборе, но не сможет обобщать результат на новые данные.

## 2 - Разбейте данные на выборки

Разбивать исходные датасет будем в два этапа.

```
Разобьем sample в следующей пропорции:
- 70% train_data
- 10% validation_data
- 20% test_data

```

In [207]:
train_data, other_data = train_test_split(df, test_size=0.30, random_state=2355555)
validation_data, test_data = train_test_split(other_data, test_size=0.70, random_state=2355555)

print('Размер тренерующей выборки:', len(train_data))
print('Размер валидационной выборки:', validation_data.shape[0])
print('Размер тестовой выборки:', test_data.shape[0])

Размер тренерующей выборки: 2249
Размер валидационной выборки: 289
Размер тестовой выборки: 676


Выделим определяющие и целевой признаки модели.

In [208]:
train_data_features = train_data.drop(['is_ultra'], axis=1)
train_data_target = train_data['is_ultra']

In [209]:
validation_data_features = validation_data.drop(['is_ultra'], axis=1)
validation_data_target = validation_data['is_ultra']

In [210]:
test_data_features = test_data.drop(['is_ultra'], axis=1)
test_data_target = test_data['is_ultra']

**Данные были разбиты на три подвыборки в пропорции 70/10/30.**

- Обучение пройдет на данных train_data
- Валидация модели на данных validation_data
- Лучшая модель по валидации будет применена на данных test_data


## 3 - Исследуйте модели

### 3.1 Decision Tree

In [211]:
%%time

best_depth = 0
best_accuracy_tree = 0

for depth in range(1, 101):
    model = DecisionTreeClassifier(random_state=2355555, max_depth=depth)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(validation_data_features)
    accuracy = accuracy_score(validation_data_target, valid_predictions)
#   print('Глубина дерева', depth,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_depth = depth
        best_accuracy_tree = accuracy
print('Лучшая глубина дерева:', best_depth,'\nЛучшая правильность ("accuracy"):', best_accuracy_tree)        

Лучшая глубина дерева: 100 
Лучшая правильность ("accuracy"): 0.7162629757785467
CPU times: user 1.29 s, sys: 19.6 ms, total: 1.31 s
Wall time: 1.37 s


**_Decision Tree:_ лучший показатель при глубине дерева 5.**

Начинаем перебирать разные гиперпараметры модели.

- ___max_depth=8___

In [212]:
%%time

best_tree_model = DecisionTreeClassifier(random_state=2355555, max_depth=8)
best_tree_model.fit(train_data_features, train_data_target)
valid_predictions = best_tree_model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Глубина дерева:', 8,'\nПравильность ("accuracy") при max_depth=8:', accuracy)

Глубина дерева: 8 
Правильность ("accuracy") при max_depth=8: 0.7716262975778547
CPU times: user 12.2 ms, sys: 3.49 ms, total: 15.7 ms
Wall time: 19.4 ms


- ___criterion="gini" --> criterion="entropy"___

In [213]:
%%time

model = DecisionTreeClassifier(random_state=2355555, max_depth=8, criterion="entropy")
model.fit(train_data_features, train_data_target)
valid_predictions = best_tree_model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Глубина дерева:', 8,'\nПравильность ("accuracy") при max_depth=8, criterion="entropy":', accuracy)

Глубина дерева: 8 
Правильность ("accuracy") при max_depth=8, criterion="entropy": 0.7716262975778547
CPU times: user 17 ms, sys: 2.56 ms, total: 19.6 ms
Wall time: 25.6 ms


Точность модели не изменилась при criterion="entropy".

- ___splitter="best" --> splitter='random'___

In [214]:
%%time

model = DecisionTreeClassifier(random_state=2355555, max_depth=8, splitter="random")
model.fit(train_data_features, train_data_target)
valid_predictions = best_tree_model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Глубина дерева:', 8,'\nПравильность ("accuracy") при max_depth=8, criterion="entropy", splitter="random":', accuracy)

Глубина дерева: 8 
Правильность ("accuracy") при max_depth=8, criterion="entropy", splitter="random": 0.7716262975778547
CPU times: user 6.89 ms, sys: 3.18 ms, total: 10.1 ms
Wall time: 11.8 ms


Точность модели не изменилась при splitter="random".

In [215]:
%%time

best_tree_model = DecisionTreeClassifier(random_state=2355555, max_depth=8)
best_tree_model.fit(train_data_features, train_data_target)
valid_predictions = best_tree_model.predict(validation_data_features)
best_accuracy_tree = accuracy_score(validation_data_target, valid_predictions)
print('Глубина дерева:', 8,'\nПравильность ("accuracy") при max_depth=8:', best_accuracy_tree)

Глубина дерева: 8 
Правильность ("accuracy") при max_depth=8: 0.7716262975778547
CPU times: user 12.3 ms, sys: 2.96 ms, total: 15.2 ms
Wall time: 16.9 ms


### 3.2 Random Forest

- подберем гиперпараметр **__n_estimators__**

In [216]:
%%time

best_estim = 0
best_accuracy = 0

for estim in range(1, 101):
    model = RandomForestClassifier(random_state=2355555, n_estimators=estim)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(validation_data_features)
    accuracy = accuracy_score(validation_data_target, valid_predictions)
    # print('Количество деревьев', estim,'Точность', accuracy)
    if accuracy > best_accuracy:
        best_estim = estim
        best_accuracy_randomforest = accuracy

print('Лучшая количество деревьев:', best_estim, 'Лучшая правильность ("accuracy"):', best_accuracy_randomforest)     

Лучшая количество деревьев: 100 Лучшая правильность ("accuracy"): 0.7889273356401384
CPU times: user 28.1 s, sys: 677 ms, total: 28.7 s
Wall time: 31.6 s


Измерим точность модели при __n_estimators=18__.

In [217]:
%%time

best_forest_model = RandomForestClassifier(random_state=2355555, n_estimators=18)
best_forest_model.fit(train_data_features, train_data_target)
valid_predictions = best_forest_model.predict(validation_data_features)
best_accuracy_randomforest = accuracy_score(validation_data_target, valid_predictions)
print('Лучшая количество деревьев: 18', 'Лучшая правильность ("accuracy"):', best_accuracy_randomforest)   

Лучшая количество деревьев: 18 Лучшая правильность ("accuracy"): 0.7923875432525952
CPU times: user 125 ms, sys: 6.07 ms, total: 131 ms
Wall time: 186 ms


- ___criterion="gini" --> criterion="entropy"___

In [218]:
%%time

model = RandomForestClassifier(random_state=2355555, n_estimators=18, criterion='entropy')
model.fit(train_data_features, train_data_target)
valid_predictions = model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Лучшая количество деревьев: 18', 'Лучшая правильность ("accuracy") при criterion="entropy":', best_accuracy)   

Лучшая количество деревьев: 18 Лучшая правильность ("accuracy") при criterion="entropy": 0
CPU times: user 143 ms, sys: 6.84 ms, total: 149 ms
Wall time: 180 ms


- **_bootstrap_ с True на False**

In [219]:
%%time

model = RandomForestClassifier(random_state=2355555, n_estimators=18, bootstrap=False)
model.fit(train_data_features, train_data_target)
valid_predictions = model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Лучшая количество деревьев: 18', 'Лучшая правильность ("accuracy") при bootstrap=False:', best_accuracy)  

Лучшая количество деревьев: 18 Лучшая правильность ("accuracy") при bootstrap=False: 0
CPU times: user 156 ms, sys: 6.04 ms, total: 162 ms
Wall time: 188 ms


Никаких изменений в метрике accruracy нет.

- **_warm_start_ с False на True**

In [220]:
%%time

model = RandomForestClassifier(random_state=2355555, n_estimators=18, warm_start=True)
model.fit(train_data_features, train_data_target)
valid_predictions = model.predict(validation_data_features)
accuracy = accuracy_score(validation_data_target, valid_predictions)
print('Лучшая количество деревьев: 18', 'Лучшая правильность ("accuracy") при warm_start=False:', best_accuracy)  

Лучшая количество деревьев: 18 Лучшая правильность ("accuracy") при warm_start=False: 0
CPU times: user 135 ms, sys: 5.7 ms, total: 141 ms
Wall time: 184 ms


Никаких изменений в метрике accruracy нет.

### 3.3 Logistic Regression

In [221]:
%%time

best_max_iter = 0
best_accuracy = 0

for max_iter in range(1, 201):
    model = LogisticRegression(random_state=2355555, max_iter=max_iter)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(validation_data_features)
    accuracy = accuracy_score(validation_data_target, valid_predictions)
    if accuracy > best_accuracy:
        best_max_iter = max_iter
        best_accuracy_logreg = accuracy

print('Лучшая количество итераций:', best_max_iter, '\nЛучшая правильность ("accuracy"):', best_accuracy_logreg)    

Лучшая количество итераций: 200 
Лучшая правильность ("accuracy"): 0.7197231833910035
CPU times: user 15.9 s, sys: 339 ms, total: 16.2 s
Wall time: 9.57 s


**Лучшее количество итераций = 89.**

In [222]:
%%time

best_reg_model = LogisticRegression(random_state=2355555)
best_reg_model.fit(train_data_features, train_data_target)
valid_predictions = best_reg_model.predict(validation_data_features)

best_accuracy_logreg = accuracy_score(validation_data_target, valid_predictions)
print('Количество итераций:', 100, '\nПравильность ("accuracy"):', best_accuracy_logreg)

Количество итераций: 100 
Правильность ("accuracy"): 0.7197231833910035
CPU times: user 111 ms, sys: 7.12 ms, total: 118 ms
Wall time: 104 ms


Увеличение в большую сторону ничего не меняет. Изменим другой гиперпараметр solver.

In [223]:
%%time

best_solver = ''
best_accuracy = 0

for solve in {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}:
    model = LogisticRegression(random_state=2355555, solver=solve)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(validation_data_features)
    accuracy = accuracy_score(validation_data_target, valid_predictions)
    print('Solver:', solve, 'Правильность:',accuracy)
    if accuracy > best_accuracy:
        best_solver= solve
        best_accuracy_logreg = accuracy
print()
print('Лучшая количество итераций:', best_solver, '\nЛучшая правильность ("accuracy"):', best_accuracy_logreg)    

Solver: newton-cg Правильность: 0.7197231833910035
Solver: sag Правильность: 0.6643598615916955
Solver: lbfgs Правильность: 0.7197231833910035
Solver: saga Правильность: 0.6643598615916955
Solver: liblinear Правильность: 0.698961937716263

Лучшая количество итераций: liblinear 
Лучшая правильность ("accuracy"): 0.698961937716263
CPU times: user 522 ms, sys: 19.3 ms, total: 541 ms
Wall time: 349 ms


**Изменим в цикле гиперпараметр _С_.**

In [224]:
%%time

best_C = 0.0
best_accuracy = 0

for float_c in [x/10 for x in range(1, 21)]:
    model = LogisticRegression(random_state=2355555, C=float_c)
    model.fit(train_data_features, train_data_target)
    valid_predictions = model.predict(validation_data_features)
    accuracy = accuracy_score(validation_data_target, valid_predictions)
    if accuracy > best_accuracy:
        best_C= float_c
        best_accuracy_logreg = accuracy

print('Лучший C:', best_C,'\nЛучшая правильность ("accuracy"):', best_accuracy_logreg) 

Лучший C: 2.0 
Лучшая правильность ("accuracy"): 0.7197231833910035
CPU times: user 2.01 s, sys: 55.8 ms, total: 2.06 s
Wall time: 1.3 s


**ВЫВОД**

???

## 4 - Проверьте модель на тестовой выборке

In [225]:
test_predictions = best_tree_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Правильность ("accuracy") модели решающего леса на тестовой выборке:', accuracy)
print('Правильность ("accuracy") модели решающего леса на тренировочной выборке:', best_accuracy_tree)
print()
test_predictions = best_forest_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Правильность ("accuracy") модели случайного леса на тестовой выборке:', accuracy)
print('Правильность ("accuracy") модели случайного леса на тренировочной выборке:', best_accuracy_randomforest)
print()
test_predictions = best_reg_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Правильность ("accuracy") модели логистической регресиии на тестовой выборке:', accuracy)
print('Правильность ("accuracy") модели логистической регресиии на тренировочной выборке:', best_accuracy_logreg) 

Правильность ("accuracy") модели решающего леса на тестовой выборке: 0.7988165680473372
Правильность ("accuracy") модели решающего леса на тренировочной выборке: 0.7716262975778547

Правильность ("accuracy") модели случайного леса на тестовой выборке: 0.7943786982248521
Правильность ("accuracy") модели случайного леса на тренировочной выборке: 0.7923875432525952

Правильность ("accuracy") модели логистической регресиии на тестовой выборке: 0.764792899408284
Правильность ("accuracy") модели логистической регресиии на тренировочной выборке: 0.7197231833910035


**ВЫВОД**

- **Случайный лес лучше, так как в нем максимально учтена проблема обученности модели, значения правильности модели (accuracy) на тренировочных и тестовых данных почти одинакова 0.79.**
```
Правильность ("accuracy") модели случайного леса на тестовой выборке: 0.7943786982248521
Правильность ("accuracy") модели случайного леса на тренировочной выборке: 0.7923875432525952
```

## 5 - Проверьте модели на адекватность

Оценить адекватность модели можно c помощью следующих метрик:

- **правильность (accuracy)** - общая точность модели, отвечает за долю верных предсказаний, считает, сколько раз test_data_target == test_predictions;
- **точность (precision)** - доля объектов, которые верно определены моделью и действительно являются верными (true positive);
- **полнота (recall)** - выявляет, какую часть дорогих объектов выделила модель.


- **accuracy**

In [226]:
test_predictions = best_forest_model.predict(test_data_features)
accuracy = accuracy_score(test_data_target, test_predictions)
print('Метрика правильности "accuracy":', accuracy)

Метрика правильности "accuracy": 0.7943786982248521


- **precision**

In [157]:
import sklearn.metrics

precision = sklearn.metrics.precision_score(test_predictions, test_data_target, pos_label=1)
print('Метрика точности "precision":', precision)

Метрика "precision": 0.5303030303030303


- **recall**

In [161]:
import sklearn.metrics

recall = sklearn.metrics.recall_score(test_data_target, test_predictions, pos_label=1)
print('Метрика полноты "recall":', recall)

Метрика полноты "recall": 0.5303030303030303


**Оценим метрики адекватности модели:**

In [160]:
print('Метрика "accuracy":', accuracy)
print('Метрика "precision":', precision)
print('Метрика "recall(полнота)":', recall)

Метрика "accuracy": 0.7943786982248521
Метрика "precision": 0.5303030303030303
Метрика "recall(полнота)": 0.5303030303030303


## Общий вывод

Наибольшую правильность (accuracy) показала модель **случайного леса** с правильностью = 0.79, которая сможет максимально правильно подобрать (посоветовать) тариф "Смарт" или "Ультра" пользователю на основе различных признаков.

Спасибо! :)